##Assignment 1: Develop a Proof-of-Concept version of an application that is querying a database to come provide an output to the user.

This colab and code was made by Raiyan M.D, and Kasper R. Haurum on the Business data science master at their 2nd semester, for Module 6: Data Engineering and MLOps.

In this assignment we had the goal of constructing a proof-of-Concept application that used either NoSQL, or SQL database, which we in this assignment decided to pick the latter.

The code used in this assignment will demonstrate the implementation of a semantic search application using SQLite, Pandas, Sentence Transformers, and Gradio. 

The application performs a similarity search based on user input and retrieves the most similar documents from a SQLite database.

The database selected for the assignment was a user creation, a SQLite database and a table called `("documents.db")`, and afterwards, filled with `20 newsgroups dataset` from sk-learn.

With this code, users can search for similar documents within the 20 newsgroups dataset using a Gradio interface, and lastly deploy it on huggingface spaces at the end.

##Creating our placeholder SQL database

We will start the assignment by installing the necessary libraries & tools required for the job. In this case we will pick SQLLite as our SQL database engine, whereas the rest is the usual tools with the trade. 

In [ ]:
!pip install sqlite3
!pip install pandas
!pip install numpy
!pip install nltk
!pip install scikit-learn
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 M

We will now create an SQLite database to store the documents and define a table schema. The database will be labeled as `("documents.db")`, and will be a placefolder database we will later fill with one of the scikit-learn datasets.
We decided this approach as creating a database of our own creation would take too long, and it was much faster to used a prepared database to do semantic search upon.

In [ ]:
import sqlite3

conn = sqlite3.connect("documents.db")

# Create a table for the documents
conn.execute("""
CREATE TABLE IF NOT EXISTS documents (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    content TEXT NOT NULL
);
""")

We will now create some sample data, and afterwards insert it into the documents.db table, and use conn.commit() to commit the changes. 

As seen, the values will be inserted into the document ID, document title, and document content. 

In [ ]:
sample_data = [
    (1, "Document 1", "This is the content of the first document."),
    (2, "Document 2", "The second document has a different content."),
    (3, "Document 3", "The content of the third document is unique."),
]

conn.executemany("""
INSERT INTO documents (id, title, content) VALUES (?, ?, ?)
""", sample_data)

conn.commit()

Now that we have a functional SQL database, we will load it into a Pandas DataFrame, and do preprocessing the content. We do this using the Natural Language Toolkit, otherwise known as NLTK. This is a ideal library to use for NLP tasks, which semantic search is.

Once we have done preprocessing on our database, we will next create embeddings using the SentenceTransformer library, using the sentence transformer ("paraphrase-MiniLM-L6-v2")

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer

# Download stopwords
import nltk
nltk.download("stopwords")
nltk.download("punkt")

# Load data from the SQLite database into a Pandas DataFrame
df = pd.read_sql_query("SELECT * FROM documents", conn)

# Preprocess the content
stop_words = set(stopwords.words("english"))

def preprocess(text):
    tokens = word_tokenize(text)
    return " ".join([word for word in tokens if word.lower() not in stop_words])

df["preprocessed_content"] = df["content"].apply(preprocess)

# Create embeddings
model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
df["embedding"] = df["preprocessed_content"].apply(lambda x: model.encode(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Now that we have a functional database complete with preprocessed content and embeddings, we want to create something which can interact with the database. In this instance, we want to create a search function. 

Since we are doing semantic search, creating a search function that could calculate cosine similarities between the query, and the document embedding, would fit perfectly in. The output would then returns the most similar documents compared to the input of query.

This logic can be seen in the code with:

`df.nlargest(n_results, "similarity")[["title", "content", "similarity"]]`

The logic behind `(n_results=3)` is because our database only have three documents, so as such we only have three results to display

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def search(query, n_results=3):
    query_preprocessed = preprocess(query)
    query_embedding = model.encode(query_preprocessed)

    similarities = [cosine_similarity([query_embedding], [doc_embedding])[0][0] for doc_embedding in df["embedding"]]
    df["similarity"] = similarities

    return df.nlargest(n_results, "similarity")[["title", "content", "similarity"]]

In order to display the results of this search function we also require a function defined that will display the results, which is the logic behind the `def display_results(results):` code

The `display_results` function takes a DataFrame, which in this case is our database converted to a Pandas dataframe, with search results and prints the title, content, and similarity score of each result in a readable format.

It iterates through each row of the input DataFrame and displays the information, without returning any value.

In [ ]:
def display_results(results):
    """
    This function takes a DataFrame containing search results and prints
    the title, content, and similarity score for each result.
    """
    print("Search Results:")
    for index, row in results.iterrows():
        print(f"Title: {row['title']}")
        print(f"Content: {row['content']}")
        print(f"Similarity: {row['similarity']:.4f}")
        print("----" * 10)


As we want to create a Gradio interface, we start by doing a `!pip install gradio` with -q to limit the tons of downloading lines.

##Preprocessing for GRadio, and launching the interface for the placeholder database

In [ ]:
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.7/129.7 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.2/286.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 24.5 MB/s eta 0:

In order to use a Gradio interface, we will have to define the output, and input structure it will utilize. This can be seen in this section:

The `search_interface(query)` function serves as a wrapper for the search function, taking a query string as input. 

It calls the search function to obtain the search results, and then iterates through the results, formatting the output as a single string. 

The output string consists of the title, content, similarity score, and a separator for each result. This can be seen here:

    for index, row in results.iterrows():
        output += f"Title: {row['title']}\n"
        output += f"Content: {row['content']}\n"
        output += f"Similarity: {row['similarity']:.4f}\n"
        output += "----" * 10 + "\n"

The function returns this formatted `output` string, which can be used for display in a user interface or console.

In [ ]:
def search_interface(query):
    results = search(query)
    output = ""
    for index, row in results.iterrows():
        output += f"Title: {row['title']}\n"
        output += f"Content: {row['content']}\n"
        output += f"Similarity: {row['similarity']:.4f}\n"
        output += "----" * 10 + "\n"
    return output

Lastly we are going to combine all this together to be used in the Gradio text interface. This will we done in the following done.

In short, it imports the Gradio library and creates two components for the user interface: 
1. an input component `(input_component = gr.inputs.Textbox(lines=2, label="Enter your query")`

and 

2. an output component. `(output_component = gr.outputs.Textbox(label="Search Results")`

In [ ]:
import gradio as gr

input_component = gr.inputs.Textbox(lines=2, label="Enter your query")
output_component = gr.outputs.Textbox(label="Search Results")

/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


We will now launch the Gradio interface with a boiler-plate template for the semantic search application.

Note: The current implementation of the semantic search application is limited by design, as it only includes three documents in the document.db database. This means that the Gradio interface can only return a maximum of three results, and the semantic search capabilities are limited as the content of the documents are one single line each. 

Seeing as our GRadio interface worked out, we will want to improve the search functionality by adding more documents to the database. However as we are lazy, we will do this by using sk-learn datasets.

In [ ]:
gr.Interface(
    fn=search_interface,
    inputs=input_component,
    outputs=output_component,
    title="GRadio test for Semantic Search",
    description="Enter a query to search for the most similar documents in the document.db database. This is our test to see if we set our Gradio interface up correctly.",
    theme="huggingface",
).launch()

/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:638: UserWarning: Cannot load huggingface. Caught Exception: The space huggingface does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

##Constructing our main database using sklearn.datasets

As described earlier, what we have done up until point is to set the field by creating a placeholder SQL database, testing preprocessing & embeddings, and finally linking it to a GRadio interface. As shown this was proven to be a successful effort, and now, we want to create a larger database as semantic search is best used on a larger corpus of text.

As we do not wish to create several new entries to our database, we will take shortcut by using one of the datasets from `sklearn.datasets`, and load the `20newsgroup dataset`. This dataset is ideal as the 20 Newsgroups dataset is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. Running a semantic search application software would thus show a more clarified result.


To being, we will construct a code that imports the `fetch_20newsgroups` function from the sklearn.datasets module and loads the 20 newsgroups dataset with all subsets ('train' and 'test') while removing headers, footers, and quotes from the data. This can be shown here:

`newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))`

It then clears the existing data from the documents table in the SQLite database by executing the `DELETE FROM documents` command. 

As it iterates through the 20 newsgroups dataset, the code creates a title for each data point by combining the target name (newsgroup category) and the index. 

It also removes leading and trailing spaces from the text content using the `strip()` function. The processed data, including the index, title, and content, is then inserted into the documents table using the:

 `INSERT INTO documents (id, title, content) VALUES (?, ?, ?) command. `

Finally, the changes are committed to the SQLite database by calling the commit() method on the connection object (conn).

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# Load the 20 newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Insert the newsgroups data into the documents table
conn.execute("DELETE FROM documents")  # Remove existing sample data

for i, (text, target) in enumerate(zip(newsgroups.data, newsgroups.target)):
    title = f"{newsgroups.target_names[target]} - {i}"
    content = text.strip()
    conn.execute("INSERT INTO documents (id, title, content) VALUES (?, ?, ?)", (i, title, content))

conn.commit()

Here we will now retrieve all the information from the `'documents'` table in the database and organizes it into a DataFrame. This is done by setting the `df` function shown here:

`df = pd.read_sql_query("SELECT * FROM documents", conn)`

It then processes the text in each document to make it easier to work with and stores the processed text in a new column. 

Lastly, it converts the processed text into embeddings, which helps the program understand the relationships between different pieces of text.

In [ ]:
df = pd.read_sql_query("SELECT * FROM documents", conn)

# Preprocess the content
df["preprocessed_content"] = df["content"].apply(preprocess)

# Create embeddings
df["embedding"] = df["preprocessed_content"].apply(lambda x: model.encode(x))

Having removed the placeholder database contents and replaced it with the 20Newgroup dataset, and also done preprocessing & embedding, we are now ready to set the GRadio interface up. As we already had constructed a data architecture on the previous iteration of a GRadio interface, we only need to use the same format as before, but with changes in title and description.

The user can in this new GRadio interface input a query or topic, and the interface will search for the most similar documents in the 20Newsgroup dataset from scikit-learn. 

In [ ]:
# Launch the Gradio interface
gr.Interface(
    fn=search_interface,
    inputs=input_component,
    outputs=output_component,
    title="The master Semantic Search application for 20Newsgroup database on SK.Learn",
    description="Enter a query, that being a subject or field, to search for the most similar documents in the 20Newsgroup database related to that prompt.",
    theme="huggingface",
).launch()

/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:638: UserWarning: Cannot load huggingface. Caught Exception: The space huggingface does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In conclusion, our Assignment 1 notebook demonstrates the implementation of a semantic search application using Gradio, SQLite, and Hugging Face's Sentence Transformers. 

By creating & connecting to an SQLite database containing a limited set of documents, we preprocess and create embeddings for each document using a pretrained `SentenceTransformer("paraphrase-MiniLM-L6-v2")` model.

The user can then search for similar documents by entering a query through the Gradio interface, which displays the most relevant results based on cosine similarity scores. 

Finally, we enhance the application by incorporating a more comprehensive dataset, the 20Newsgroup dataset from scikit-learn, to showcase the potential of semantic search in finding relevant information across a larger collection of documents.